<a href="https://colab.research.google.com/github/darkfapper/Pandoras_Box/blob/main/Experimentos_Double_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install DoubleML numba econml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 15.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from doubleml.double_ml_plr import DoubleMLPLR
from sklearn.preprocessing import Binarizer
import xgboost as xgb
from sklearn.base import clone
# suppress scientific notation by setting float_format
pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(suppress=True)

In [ ]:
# Load the housing data into a pandas dataframe
train_data = pd.read_csv('/content/sample_data/california_housing_train.csv')
test_data = pd.read_csv('/content/sample_data/california_housing_test.csv')

# housing_data = train_data.append(test_data)
housing_data = pd.concat([train_data,test_data],axis = 0)
housing_data["median_house_value"] = np.log(housing_data["median_house_value"])
housing_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.00,5612.00,1283.00,1015.00,472.00,1.49,11.11
1,-114.47,34.40,19.00,7650.00,1901.00,1129.00,463.00,1.82,11.29
2,-114.56,33.69,17.00,720.00,174.00,333.00,117.00,1.65,11.36
3,-114.57,33.64,14.00,1501.00,337.00,515.00,226.00,3.19,11.20
4,-114.57,33.57,20.00,1454.00,326.00,624.00,262.00,1.93,11.09
...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23.00,1450.00,642.00,1258.00,607.00,1.18,12.32
2996,-118.14,34.06,27.00,5257.00,1082.00,3496.00,1036.00,3.39,12.38
2997,-119.70,36.30,10.00,956.00,201.00,693.00,220.00,2.29,11.03
2998,-117.12,34.10,40.00,96.00,14.00,46.00,14.00,3.27,12.00


In [ ]:
housing_data['residential_bedrooms'] = Binarizer(threshold=25).fit_transform(housing_data['total_bedrooms'].values.reshape(-1,1))

In [ ]:

# Define the treatment, outcome, and confounding variables
treatment = ['total_bedrooms']
outcome = 'median_house_value'
confounders = ['housing_median_age', 'total_rooms', 'population', 'median_income',"households"]

# Create the DoubleML data object
data = DoubleMLData(data=housing_data, y_col=outcome, d_cols=treatment,x_cols=confounders)

# Define the machine learning models to be used for the treatment and outcome models

model = xgb.XGBRegressor()

#model = LinearRegression()

treatment_model = clone(model)
outcome_model = clone(model)

# Create the DoubleMLPLR object using the data and models
dml_plr = DoubleMLPLR(obj_dml_data=data, ml_l=treatment_model, ml_m=outcome_model,n_rep=1,dml_procedure="dml2")

# Fit the DoubleMLPLR model
print(dml_plr.fit())


================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: median_house_value
Treatment variable(s): ['total_bedrooms']
Covariates: ['housing_median_age', 'total_rooms', 'population', 'median_income', 'households']
Instrument variable(s): None
No. Observations: 20000

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
  

In [ ]:
dml_plr.summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
total_bedrooms,0.00,0.00,7.38,0.00,0.00,0.00


In [ ]:
from sklearn.metrics import r2_score
def doubleml_r2(y_true, y_pred):
    subset = np.logical_not(np.isnan(y_true))
    return r2_score(y_true[subset], y_pred[subset])
dml_plr.evaluate_learners(metric=doubleml_r2)

{'ml_l': array([[0.62391398]]), 'ml_m': array([[0.97263564]])}

In [ ]:
# Main imports
from econml.dml import DML, LinearDML, SparseLinearDML, CausalForestDML
import econml

/usr/local/lib/python3.10/dist-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/usr/local/lib/python3.10/dist-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swap_size=100, num_

In [ ]:
from sklearn.linear_model import LinearRegression
Y_train = housing_data[['median_house_value']]
# T_train = housing_data[['total_bedrooms','residential_bedrooms']]
T_train = housing_data['total_bedrooms']
# T_train = housing_data['residential_bedrooms']
X_train = housing_data[['housing_median_age', 'total_rooms', 'population', 'median_income',"households"]]

model = xgb.XGBRegressor()

model_y = clone(model)

model_t = clone(model)

est = LinearDML(model_y=model_y,
                model_t=model_t,
                cv=5,
                # model_final = LinearRegression(fit_intercept=False),
                random_state=123,linear_first_stages=False)
print(est.fit(Y_train, T_train, X=X_train))

In [ ]:
est.summary

<bound method LinearModelFinalCateEstimatorMixin.summary of <econml.dml.dml.LinearDML object at 0x7ca0b87ff8b0>>

In [ ]:
est.const_marginal_ate(X_train)

array([0.00097096])

In [ ]:
from sklearn.model_selection import train_test_split , cross_val_predict , cross_val_score
## Exemplo retirado de https://matheusfacure.github.io/python-causality-handbook/22-Debiased-Orthogonal-Machine-Learning.html

T = 'total_bedrooms'
y = 'median_house_value'
X = ['housing_median_age', 'total_rooms', 'population', 'median_income',"households"]

# train, test = train_test_split(housing_data, test_size=0.2, random_state=123456)

train = housing_data.copy()

debias_m = xgb.XGBRegressor()

# T = housing_data[treatment]
# Y = housing_data[outcome]
# X = housing_data[confounders]
# T_train, T_test, Y_train, Y_test, X_train, X_test = train_test_split(T, Y, X, test_size=0.2, random_state=123456)

In [ ]:
train_pred = train.assign(price_res =  train[T] -
                          cross_val_predict(debias_m, train[X], train[T], cv=5)
                          + train[T].mean()) # add mu_t for visualization.

In [ ]:
denoise_m = xgb.XGBRegressor()

train_pred = train_pred.assign(sales_res =  train[y] -
                               cross_val_predict(denoise_m, train[X], train[y], cv=5)
                               + train[y].mean())

In [ ]:
import statsmodels.formula.api as smf
final_model = smf.ols(formula='sales_res ~ price_res', data=train_pred).fit()
final_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              sales_res   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     93.48
Date:                Mon, 08 Jan 2024   Prob (F-statistic):           4.57e-22
Time:                        00:02:51   Log-Likelihood:                -8045.4
No. Observations:               20000   AIC:                         1.609e+04
Df Residuals:                   19998   BIC:                         1.611e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.9149      0.018    667.921      0.000      11.880      11.950
price_res      0.0003   3.28e-05      9.669      0.000       0.000       0.000
==============================================================================
Omnibus:                      838.607   Durbin-Watson:                   1.461
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2700.387
Skew:                          -0.077   Prob(JB):                         0.00
Kurtosis:                       4.793   Cond. No.                     3.79e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.79e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
np.e ** 0.0003

1.0003000450045003